In [17]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os


from src.data_loader import *

In [20]:
class StockDataset(Dataset):
    def __init__(self, path, date_type, interval = 'day', ticker = None, transform=None):
        self.ticker = ticker
        self.transform = transform
        self.df = DataGenerator(date_type, interval).data_search(ticker = self.ticker).values.astype(np.np.float32)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df[idx]
        if self.transform is not None:
            data = self.transform(data)
        return data

In [21]:
dataset = StockDataset(path = './Dataset',date_type = 'csv', interval='day', ticker = 'GOOG' )
dataloader = DataLoader(dataset=dataset,
                        batch_size=1,
                        shuffle=True,
                        drop_last=False)

파일에서 CSV 데이터 불러오기 성공


In [22]:
for epoch in range(2):
    print(f"epoch : {epoch} ")
    for batch in dataloader:
        print(batch)

epoch : 0 


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object